Part 1

In [11]:
import os
import re
import random
from collections import Counter, defaultdict
from copy import copy

import numpy as np
import pandas as pd
from nltk import everygrams
from nltk.tokenize import RegexpTokenizer
from tqdm.notebook import tqdm

In [2]:
if not os.path.exists("/content/corpora.zip"):
    !wget -q https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip
    !unzip -oq corpora.zip

In [3]:
FILES = ["AnnaKarenina.txt", "WarAndPeace.txt"]

In [20]:
corpus = []
for filename in FILES:
    with open(filename, "r") as fin:
        corpus += fin.readlines()
corpus = " ".join(corpus)

In [22]:
ALPHABET = " абвгдежзийклмнопрстуфхцчшщъыьэюя"

In [24]:
def tokenize(text, alphabet=ALPHABET, tokenizer=RegexpTokenizer(r"\w+")):
    text = text.lower()
    
    text = "".join([c for c in text if c in alphabet])
    return " ".join(tokenizer.tokenize(text))


def get_ngram_freqs(text, n_gram=1):
    if n_gram > 1:
        text = [
            "".join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)
        ]
    freqs = {
        k: v / len(text)
        for k, v in Counter(text).items()
        if v > 0  # remove zeros, because why do we need them?
    }
    return freqs


def generate_mapping(freqs):
    original = list(freqs.keys())
    replacements = np.random.choice(original, replace=False, size=len(freqs))
    mapping = {
        original_char: replacement_char
        for original_char, replacement_char in zip(original, replacements)
    }
    return mapping


def apply_mapping(text, mapping):
    return "".join([mapping.get(c, "ь") for c in text])


def get_reverse_mapping(corpus_freqs, text_freqs):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)

    reverse_mapping = {}
    for text_char, text_freq in text_freqs_sorted:
        min_diff = 1.0  # maximum possible frequency
        best_char = None
        for corpus_char, corpus_freq in corpus_freqs_sorted:
            diff = abs(corpus_freq - text_freq)
            if diff < min_diff:
                best_char = corpus_char
                min_diff = diff

        reverse_mapping[text_char] = best_char
        corpus_freqs_sorted = [
            (char, freq) for char, freq in corpus_freqs_sorted if char != best_char
        ]

    return reverse_mapping


def character_accuracy(text1, text2):
    assert len(text1) == len(text2)
    matching_chars = sum((c1 == c2) for c1, c2 in zip(text1, text2))
    return matching_chars / len(text1)

In [25]:
tokenized_corpus = tokenize(corpus)
corpus_freqs = get_ngram_freqs(tokenized_corpus, n_gram=1)
mapping = generate_mapping(corpus_freqs)

In [26]:
text = """
    Как ныне сбирается вещий Олег\n
    Отмстить неразумным хозарам:\n
    Их села и нивы за буйный набег\n
    Обрек он мечам и пожарам;\n
    С дружиной своей, в цареградской броне,\n
    Князь по полю едет на верном коне.\n

    Из темного леса навстречу ему\n
    Идет вдохновенный кудесник,\n
    Покорный Перуну старик одному,\n
    Заветов грядущего вестник,\n
    В мольбах и гаданьях проведший весь век.\n
    И к мудрому старцу подъехал Олег.\n

    «Скажи мне, кудесник, любимец богов,\n
    Что сбудется в жизни со мною?\n
    И скоро ль, на радость соседей-врагов,\n
    Могильной засыплюсь землею?\n
    Открой мне всю правду, не бойся меня:\n
    В награду любого возьмешь ты коня».\n
"""

In [27]:
tokenized_text = tokenize(" ".join(text.split("\n")))
encoded_text = apply_mapping(tokenized_text, mapping)
text_freqs = get_ngram_freqs(encoded_text)
tokenized_text

'как ныне сбирается вещий олег отмстить неразумным хозарам их села и нивы за буйный набег обрек он мечам и пожарам с дружиной своей в цареградской броне князь по полю едет на верном коне из темного леса навстречу ему идет вдохновенный кудесник покорный перуну старик одному заветов грядущего вестник в мольбах и гаданьях проведший весь век и к мудрому старцу подъехал олег скажи мне кудесник любимец богов что сбудется в жизни со мною и скоро ль на радость соседейврагов могильной засыплюсь землею открой мне всю правду не бойся меня в награду любого возьмешь ты коня'

In [28]:
encoded_text

'йрйкшяшык абирыв ькэызбюкцтылкцвч вбвжкшыирпдчшячкуцприрчкбук ытркбкшбэякпркадюшяюкшраылкцаиыйкцшкчыгрчкбкоцщрирчк книдщбшцюк эцыюкэкериылирн йцюкаицшыкйшьпжкоцкоцтфкынывкшркэыишцчкйцшыкбпквычшцлцкты ркшрэ виыгдкычдкбнывкэнцушцэышшяюкйдны шбйкоцйцишяюкоыидшдк врибйкцншцчдкпрэывцэклиьндзылцкэы вшбйкэкчцтжарукбклрнршжьукоицэынъбюкэы жкэыйкбкйкчдницчдк вриедкоцнмыурткцтылк йрщбкчшыкйдны шбйктфабчыекацлцэкгвцк адныв ькэкщбпшбк цкчшцфкбк йцицктжкшркирнц вжк ц ыныюэирлцэкчцлбтжшцюкпр яотф жкпычтыфкцвйицюкчшыкэ фкоирэндкшыкацю ькчышькэкшрлирндктфацлцкэцпжчыъжквякйцшь'

In [29]:
reverse_mapping = get_reverse_mapping(corpus_freqs, text_freqs)
decoded_text = apply_mapping(encoded_text, reverse_mapping)
decoded_text

'кик ачае тгвлиемтж сецвп ояеь омртмвмы аелибурачр шобилир вш теяи в авсч би гупачп аигеь оглек оа реюир в зохилир т длухваоп тсоеп с эилеьлидткоп глоае кажбы зо зояй едем аи селаор коае вб мераоьо яети аистмлеюу еру вдем сдошаосеаачп кудетавк зоколачп зелуау тмилвк одаору бисемос ьлждуцеьо сетмавк с рояыгиш в ьидиаыжш злоседщвп сеты сек в к рудлору тмилэу зодфешия ояеь ткихв рае кудетавк яйгвреэ гоьос юмо тгудемтж с хвбав то раой в тколо яы аи лидотмы тотедепслиьос роьвяыаоп битчзяйты беряей омклоп рае стй злисду ае гоптж реаж с аиьлиду яйгоьо собырещы мч коаж'

In [30]:
character_accuracy(tokenized_text, decoded_text)

0.42226148409893993

Part 2

In [31]:
corpus_freqs_bigram = get_ngram_freqs(tokenized_corpus, n_gram=2)
text_freqs_bigram = get_ngram_freqs(encoded_text, n_gram=2)

corpus_freqs_sorted = sorted(
    corpus_freqs_bigram.items(), key=lambda x: x[1], reverse=True
)
text_freqs_sorted = sorted(text_freqs_bigram.items(), key=lambda x: x[1], reverse=True)

In [32]:
def get_reverse_mapping_ngram(corpus_freqs, text_freqs, n_gram=1):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)

    # Iterate over n-grams starting from the most frequent. On each iteration
    # we take into account already decoded symbols:
    reverse_mapping = {}
    for i, (text_ngram, text_freq) in enumerate(text_freqs_sorted):
        filtered_freqs = copy(corpus_freqs_sorted)

        for j in range(n_gram):
            if text_ngram[j] in reverse_mapping:
                filtered_freqs = [
                    (ngram, freq)
                    for ngram, freq in filtered_freqs
                    if ngram[j] == reverse_mapping[text_ngram[j]]
                ]

        min_diff = 1.0  # maximum possible frequency
        best_ngram = None
        for ngram, freq in filtered_freqs:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_ngram = ngram
                min_diff = diff

        for j in range(n_gram):
            if text_ngram[j] not in reverse_mapping:
                reverse_mapping[text_ngram[j]] = best_ngram[j]

    return reverse_mapping

In [33]:
reverse_mapping_bigram = get_reverse_mapping_ngram(
    corpus_freqs_bigram, text_freqs_bigram, n_gram=2
)
decoded_text = apply_mapping(encoded_text, reverse_mapping_bigram)
decoded_text

'тсто к по вн спи во пентонипоонин ини о п сотн кнолнос снонло писоно н коосовтт кто свпоонв птон онпеснононнас сно о  тан нто  нпто ожс по с  тнтов н пот во онноннилоп пио со п  ннотн понооипн ноноип со с  и петопнтон пио  нл н п  ктотт п  нтоннтн  ктонп т то ис нтон  ннтоос пин оо в тепоно п и нто онни вслоноос с  влон н п янто п  о птонотонт  ннто ис жтонн яплсионипоо тсанон потт п  нтоилвннпжовнон оеино вт пи во оано но нон нлоно тн нои о со с н и о н п пт  сон оннони  нтоос книл  оопниплонит нтон по  лон с  то повнт вонп во о со с тоилвноно но нпя оикотн в'

In [34]:
character_accuracy(tokenized_text, decoded_text)

0.0

Part 3

Текст, разбитый на биграммы — это, по сути, марковская цепь. Частота биграмм — вероятность перехода между состояниями цепи.

Для обучения перестановки будем считать вероятность порождения именно такого текста как произведение вероятностей всех биграмм, в него входящих.

Всего перестановок очень много, поэтому будем использовать жадный алгоритм, основанный на идее MCMC-сэмплирования.

Алгоритм:
1. Инициализируем перестановки, восстанавливаем текст и вычисляем логарифм правдоподобия $p_{current}$.
2. Меняем местами пару символов для перестановки.
3. Восстанавливаем текст с новой перестановкой и вычисляем $p_{proposed}$.
4. Принимаем новую перестановку с "вероятностью" $\displaystyle p_{accept} = \frac{p_{proposed}}{p_{current}}$.
5. Возвращаемся к пункту 2 и повторяем цикл.

Мы можем уйти немного не туда и застрять не в том максимуме, поэтому на каждой итерации алгоритма будем делать много попыток и брать в качестве окончательного результата лучшую из них:

In [35]:
def get_ngram_freqs_smoothed(text, n_gram=2):
    vocab_size = len(set(text)) ** n_gram
    if n_gram > 1:
        text = [
            "".join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)
        ]
    freqs = {
        k: (v + 1) / (len(text) + vocab_size)  # сглаживаем, чтобы не было нулей
        for k, v in Counter(text).items()
    }
    return freqs


def get_text_proba(text, mapping, freqs, n_gram=2, alphabet=ALPHABET):
    decoded_text = apply_mapping(text, mapping)
    log_proba = 0.0
    for i in range(len(decoded_text) - n_gram):
        ngram = decoded_text[i : i + n_gram]
        ngram_proba = freqs.get(
            ngram, 1 / (len(text) + len(alphabet) ** n_gram)
        )  # сглаживаем, чтобы не было нулей
        log_proba += np.log(ngram_proba)
    return log_proba


def get_reverse_mapping_mcmc(
    encoded_text,
    alphabet_encoded,
    alphabet_corpus,
    freqs_corpus,
    n_iters=10000,
    n_trials=10,
    n_gram=2,
):
    accept_count = 0
    best_reverse_mapping = None
    all_mappings = []
    best_log_likelihood = -np.inf

    for trial in tqdm(range(n_trials), leave=False, position=0, total=n_trials):
        alphabet_encoded = list(alphabet_encoded)
        alphabet_iter = list(alphabet_corpus)
        reverse_mapping = {
            k: v
            for k, v in zip(alphabet_encoded, alphabet_iter[: len(alphabet_encoded)])
        }
        log_proba_current = get_text_proba(
            encoded_text, reverse_mapping, freqs_corpus, n_gram=n_gram
        )

        for i in range(n_iters):
            alphabet_proposal = alphabet_iter[:]
            idx1, idx2 = np.random.choice(len(alphabet_proposal), replace=False, size=2)
            alphabet_proposal[idx1], alphabet_proposal[idx2] = (
                alphabet_proposal[idx2],
                alphabet_proposal[idx1],
            )
            reverse_mapping_proposal = {
                k: v
                for k, v in zip(
                    alphabet_encoded, alphabet_proposal[: len(alphabet_encoded)]
                )
            }
            log_proba_proposal = get_text_proba(
                encoded_text, reverse_mapping_proposal, freqs_corpus, n_gram=n_gram
            )

            p_accept = np.exp(log_proba_proposal - log_proba_current)

            if p_accept > np.random.rand():
                accept_count += 1
                alphabet_iter = alphabet_proposal
                log_proba_current = log_proba_proposal
                reverse_mapping = reverse_mapping_proposal

        if log_proba_current > best_log_likelihood:
            best_log_likelihood = log_proba_current
            best_reverse_mapping = reverse_mapping

        all_mappings.append(reverse_mapping)

    print(f"Best likelihood: {best_log_likelihood}")
    print(f"Accept ratio: {accept_count / (n_iters * n_trials)}")
    return best_reverse_mapping

In [36]:
freqs_corpus = get_ngram_freqs_smoothed(tokenized_corpus, n_gram=2)

In [37]:
best_reverse_mapping = get_reverse_mapping_mcmc(
    encoded_text,
    alphabet_encoded=ALPHABET,
    alphabet_corpus=ALPHABET,
    freqs_corpus=freqs_corpus,
)

  0%|          | 0/10 [00:00<?, ?it/s]

Best likelihood: -3176.681208804238
Accept ratio: 0.01597


In [38]:
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
decoded_text

'как ныне сбирается веший олег отмстить неразумным хозарам их села и нивы за буйный набег обрек он меэам и пожарам с дружиной своей в чареградской броне князь по полю едет на верном коне из темного леса навстреэу ему идет вдохновенный кудесник покорный перуну старик одному заветов грядушего вестник в мольбах и гаданьях проведъий весь век и к мудрому старчу подцехал олег скажи мне кудесник любимеч богов это сбудется в жизни со мною и скоро ль на радость соседейврагов могильной засыплюсь землею открой мне всю правду не бойся меня в награду любого возьмеъь ты коня'

In [39]:
character_accuracy(tokenized_text, decoded_text)

0.980565371024735

Part 4

In [40]:
message = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [41]:
message_freqs = get_ngram_freqs(message, n_gram=1)

In [42]:
corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
message_freqs_sorted = sorted(message_freqs.items(), key=lambda x: x[1], reverse=True)

alphabet_corpus = "".join([c for c, _ in corpus_freqs_sorted])
alphabet_message = "".join([c for c, _ in message_freqs_sorted])
alphabet_corpus, alphabet_message

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', '↹←⇛↟⇒↝⇴↨⇠⇯↷⇌⇊⇞⇈⇷↤↳↾↙⇙↲↞⇆⇰⇸↘⇏')

In [43]:
freqs_corpus = get_ngram_freqs_smoothed(tokenized_corpus, n_gram=2)

In [44]:
best_reverse_mapping = get_reverse_mapping_mcmc(
    message,
    alphabet_encoded=alphabet_message,
    alphabet_corpus=alphabet_corpus,
    freqs_corpus=freqs_corpus,
    n_iters=10000,
    n_trials=50,
)

  0%|          | 0/50 [00:00<?, ?it/s]

Best likelihood: -1231.2547551943435
Accept ratio: 0.04388


In [45]:
encoded_message = apply_mapping(message, best_reverse_mapping)
encoded_message

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаъ'

In [46]:
original_message = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"

In [47]:
character_accuracy(original_message, encoded_message)

0.9347826086956522